# Week3_4 Assignment

## [BASIC](#Basic) 
- Encoder & Decoder Layer 코드를 직접 필사하고 각 함수에 주석을 달 수 있다. 

## [CHALLENGE](#Challenge)
- 텐서의 크기(shape)를 계산할 수 있다. 

## [ADVANCED](#Advanced)
- 완성된 transformer 모델의 모든 학습 가능한 파라미터 이름과 크기(shape)를 출력할 수 있다.

### Informs
이번 과제에서는 "[Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)"의 코드를 필사해본다.   
"Annotated Transformer"는 "Attention is all you need" 논문에서 제안한 transformer 모델을 pytorch 라이브러리로 직접 구현한다.   
코드 필사를 통해 다음을 배울 수 있다.    
- Encoder, Decoder 구조
- Attention Mechanism
- "residual connection", "layer normalization" 등의 구조 

코드 필사를 시작하기 앞서, transformer 모델의 최종 구조를 살펴보자.    

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_full.png?raw=true" width="500" align="center"/>

최종 모델은 `EncoderDecoder()` 클래스에 여러 인스턴스를 생성자의 입력 파라미터로 넣어 생성한다.    
앞으로 우리는 `EncoderDecoder()` 클래스와 같은 여러 클래스들을 구현하고 연결할 것이다. 따라서 대략적인 클래스간의 관계를 살펴보고 이해한다면 보다 큰 그림을 가지고 코드 필사를 할 수 있을 것이다. 

Transformer 모델은 크게 4가지 클래스로 구현된다.    
- Frame
    - frame 역할을 하는 `EncoderDecoder` 클래스
- Input Embedding & Encoding
    - 입력값을 벡터화하는 `Embeddings`, `PositionalEncoding`
- Encoder & Decoder
    - 각 6개 layer를 갖고 있는 `Encoder`, `Decoder`
    - layer 1층을 구현한 `EncoderLayer`, `DecoderLayer`
- Sublayer
    - `EncoderLayer`, `DecoderLayer` 내부에서 사용되는 Sublayer 클래스인 `MultiHeadAttiontion`, `PositionwiseFeedForward`
    - Sublayer 클래스들을 연결하는 `SublayerConnection`
    
아래 좌측 도식에서 각 클래스의 색상은 아래 우측 도식(transformer 구조)의 색상과 맵핑되어 있다.    
각 클래스의 역할과 클래스 간 연결 관계를 생각하면서 transformer를 코드로 구현해보자.   


<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_map.png?raw=true" width="400" height="400" align="left"/>
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_transformer.png?raw=true" width="300" height="400" align="right"/>



In [ ]:
import os
import sys
import pandas as pd
import numpy as np 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import math, copy, time
import random

## Basic

### Frame
- `EncoderDecoder`

아래 도식은 `EncoderDecoder` 클래스의 `forward()`, `encode()`, `decode()` 메소드를 도식화 한 것이다.    
 
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_encoderdecoder.png?raw=true" width=500>


- `Generator`

In [ ]:
class EncoderDecoder(nn.Module):
    ''' Standard Encoder-Decoder 모델이다. 다른 모델들과 이 모델에 쓰이는 Base이다.'''
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
    
    def forward(self, src, tgt, src_mask, tgt_mask):
        # masked source와 target sequences를 받아와 진행
        return self.decode(self.encode(src, src_mask), src_mask,
                           tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [ ]:
class Generator(nn.Module):
    '''
    Decoder의 output을 Transformer의 최종 output이 되도록 거쳐가는 추가적인 layer
    Decoder의 output은 (n_batch * seq_len * d_model)의 matrix
    standard linear와 softmax를 정의함
    '''
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab) # input에 대해 y = wx+b 형태의 선형 변환을 수행
    
    def forward(self, x):
        # 실제 token으로 변환할 수 있도록 차원을 수정
        return F.log_softmax(self.proj(x), dim=1)

### Encoder
- `Encoder`
- `EncoderLayer`
- `SublayerConnection`
- Reference
    - Layer Normalization
        - [한국어 설명](https://yonghyuc.wordpress.com/2020/03/04/batch-norm-vs-layer-norm/)
        - [torch official docs](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html)
    - Residual Connection
        - [한국어 설명](https://itrepo.tistory.com/36)
    - pytorch ModuleList
        - [torch official docs](https://pytorch.org/docs/1.9.1/generated/torch.nn.ModuleList.html)


In [ ]:
def clones(module, N):
    '''N개의 동일한 layer를 만드는 함수'''
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
class Encoder(nn.Module):
    ''' Core encoder는 N개의 layer들의 stack이다.'''
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        # Layer Normalization 수행
        self.norm = LayerNorm(layer.size)
    
    def forward(self, x, mask):
        # Input(그리고 mask)를 차례대로 각각 Layer에 전달한다.
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [ ]:
class LayerNorm(nn.Module):
    '''
    이미지 인식에서 쓰이는 residual connection을 2개의 sub-layer에 각각 적용한다.
    그 후 layer normalization이 수행된다.
    '''
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps
    
    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x -mean) / (std + self.eps) + self.b_2

In [ ]:
class SublayerConnection(nn.Module):
    '''
    Residual connection 이후에 Layer Norm이 실행됨
    '''
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x, sublayer):
        # 같은 크기를 같는 sub layer에 residual connection을 적용
        return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
class EncoderLayer(nn.Module):
    # Encoder는 Self-attention과 Feed forward로 구성되어 있다.
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size
        
    def forward(self, x, mask):
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

### Decoder
- `Decoder`
- `DecoderLayer`

In [ ]:
class Decoder(nn.Module):
    # Decoder도 6개의 동일한 Layer로 구성되어 있다.
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
    
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [ ]:
class DecoderLayer(nn.Module):
    '''
    Decoder 는 3개의 sub-layer로 구성됩니다.
    Self-attention
    Multi-head attention
    Feed forward 
    '''
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
    
    def forward(self, x, memory, src_mask, tgt_mask):
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

### Sublayer
- `attention` 함수

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_attention.png?raw=true" width="500" align="center"/>  

- `MultiHeadedAttention`
- `PositionwiseFeedForward`

### Challenge


### Q1. 위 도식에 따라 `score`, `p_attn`, `attention` 을 구하라 

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
    '''
    Scaled Dot Product Attention을 계산한다. 
    '''
    d_k = query.size(-1)
    # scaling 
    scores = torch.matmul(query, key.transpose(-2, -1)) \
            / math.sqrt(d_k) 
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1) # Probability로 변환시켜 줌
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn

###Q2. query, key, value가 모두 (m, d_k) shape의 matrix라고 가정할 때, `score`, `p_attn`, `attention`의 shape을 각각 구하라
- score : (m, m)
- p_attn : (m, m)
- attention : (m, d_k)

### (아래의 **Q3을 먼저 풀고 돌아오세요**) Q4.  query, key, value가 모두 (12, 8, 1, 64) shape의 tensor라고 가정할 때 , `score`, `p_attn`, `attention`의 shape을 각각 구하라

- score : torch.Size([12, 8, 1, 1])
- p_attn : torch.Size([12, 8, 1, 1])
- attention : torch.Size([12, 8, 1, 64])

- `MultiHeadedAttention`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_multihead.png?raw=true" width="300" align="center"/>  

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        # model dimension과 heads의 수를 받는다.
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # 논문 저자는 Key의 dimension과 value의 dimension은 동일하다고 가정합니다.
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)
    
    def forward(self, query, key, value, mask=None):
        if mask is not None:
            # Masking을 해야하는 조건이 주어진다면 Masking을 수행한다.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # d_model로 부터 batch 안에 linear projection을 수행한다. 결과로 h * d_k
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for l, x in zip(self.linears, (query, key, value))]

        # batch 안에서 projected vector 전체에 attention을 적용한다.
        x, self.attn = attention(query, key, value, mask=mask,
                                dropout=self.dropout)
        
        # view()를 이용하여 concat을 수행하고, 마지막 linear를 적용합니다.
        x = x.transpose(1, 2).contiguous() \
            .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

### Q3.  query, key, value가 모두 (12, 512) shape의 matrix이고, h 값이 8 이라고 가정할 때, 아래 값의 shape을 각각 구하라

- `d_k` (d_k = d_model // h) : 64
- `nn.Linear(d_model, d_model)(query)` : torch.Size([12, 512])
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k)` : torch.Size([12, 1, 8, 64])
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k).transpose(1,2)` : torch.Size([12, 8, 1, 64])

- `PositionwiseFeedForward`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pwff.png?raw=true" width="300" align="center"/>  

In [ ]:
class PositionwiseFeedForward(nn.Module):
    # FFN 방정식을 수행합니다
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model,d_ff) 
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

### Input Embedding & Encoding
- `Embeddings`
    - [pytorch official docs](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [ ]:
class Embeddings(nn.Module):
    #
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        # lut = Look up table : key-value 구조로 저장되어 있음. 여기선 vocab이 들어가게 됨
        # 장점은 데이터 차지 용량 감소, 성능 증가, 데이터 유지보수가 쉬움.
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model
    
    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

- `PositionalEncoding`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pe.png?raw=true" width="500" align="center"/>  

- `position` 변수 설명
    - 모든 position (=최대 토큰 개수)의 값을 갖고 있는 matrix
- `div_term` 변수 설명

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_div.png?raw=true" width="500" align="center"/>  
- `Embedding` + `Encoding` 도식화 

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_emb_enc.png?raw=true" width="400" align="center"/>  


In [ ]:
class PositionalEncoding(nn.Module):
    # Positional Encoding을 하는 클래스입니다.
    def __init__(self, d_model, dropout, max_len = 5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # 로그 차원에서 한번에 PE를 계산합니다.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        # 짝수는 Sine, 홀수는 Cosine을 이용하는 것으로 추정
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)],
                         require_grad=False)
        return self.dropout(x)

### Q4.  max_len이 512이고, d_model이 512라고 가정할 때, `position`과 `div_term`의 shape을 구하라

- `position` : torch.Size([512, 1])
- `div_term` : torch.Size([256])
- `position * div_term` : torch.Size([512, 256])

### Advanced

### Finally Build Model
- Xavier Initialization
    - [한국어 자료](https://huangdi.tistory.com/8)
    - [pytorch official docs](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.xavier_uniform_)

In [ ]:
def make_model(src_vocab, tgt_vocab, 
               N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    c = copy.deepcopy # 내부의 객체들까지 모두 새롭게 Copy
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
            
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p) # 가중치 초기화 과정
    return model

In [ ]:
model = make_model(10,10)

### Q5. 위 코드로 만든 모델의 모든 파라미터의 이름과 크기 (shape) 을 출력하라

In [ ]:
for name, param in model.named_parameters():
    result = f"Parameter's Name : {name}, // Parameter's Shape : {param.shape}"
    print(result)

Parameter's Name : encoder.layers.0.self_attn.linears.0.weight, // Parameter's Shape : torch.Size([512, 512])
Parameter's Name : encoder.layers.0.self_attn.linears.0.bias, // Parameter's Shape : torch.Size([512])
Parameter's Name : encoder.layers.0.self_attn.linears.1.weight, // Parameter's Shape : torch.Size([512, 512])
Parameter's Name : encoder.layers.0.self_attn.linears.1.bias, // Parameter's Shape : torch.Size([512])
Parameter's Name : encoder.layers.0.self_attn.linears.2.weight, // Parameter's Shape : torch.Size([512, 512])
Parameter's Name : encoder.layers.0.self_attn.linears.2.bias, // Parameter's Shape : torch.Size([512])
Parameter's Name : encoder.layers.0.self_attn.linears.3.weight, // Parameter's Shape : torch.Size([512, 512])
Parameter's Name : encoder.layers.0.self_attn.linears.3.bias, // Parameter's Shape : torch.Size([512])
Parameter's Name : encoder.layers.0.feed_forward.w_1.weight, // Parameter's Shape : torch.Size([2048, 512])
Parameter's Name : encoder.layers.0.feed